In [1]:
import urllib
import os
import pathlib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [4]:
DATA_ROOT = '../data/'
READ_DIR = os.path.join(DATA_ROOT, 'raw/bos_analyze')
WRITE_DIR = os.path.join(DATA_ROOT, 'interim')
readfile_prop = os.path.join(READ_DIR, 'property_assess/fy19fullpropassess.csv')
readfile_sam = os.path.join(READ_DIR, 'SAMaddresses.csv')

print(readfile_prop, readfile_sam)

../data/raw/bos_analyze/property_assess/fy19fullpropassess.csv ../data/raw/bos_analyze/SAMaddresses.csv


In [5]:
df_sam = pd.read_csv(readfile_sam, dtype=str)
df_prop = pd.read_csv(readfile_prop, dtype=str)

In [7]:
print(df_sam.info())
print()
print(df_prop.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398702 entries, 0 to 398701
Data columns (total 27 columns):
X                       398702 non-null object
Y                       398702 non-null object
SAM_ADDRESS_ID          398702 non-null object
RELATIONSHIP_TYPE       398702 non-null object
BUILDING_ID             398702 non-null object
FULL_ADDRESS            398702 non-null object
STREET_NUMBER           398702 non-null object
IS_RANGE                398702 non-null object
RANGE_FROM              26175 non-null object
RANGE_TO                26175 non-null object
UNIT                    270100 non-null object
FULL_STREET_NAME        398702 non-null object
STREET_ID               398702 non-null object
STREET_PREFIX           398702 non-null object
STREET_BODY             398702 non-null object
STREET_SUFFIX_ABBR      394519 non-null object
STREET_FULL_SUFFIX      394519 non-null object
STREET_SUFFIX_DIR       398702 non-null object
STREET_NUMBER_SORT      398702 non-null objec

In [8]:
df_sam.head()

,X,Y,SAM_ADDRESS_ID,RELATIONSHIP_TYPE,BUILDING_ID,FULL_ADDRESS,STREET_NUMBER,IS_RANGE,RANGE_FROM,RANGE_TO,...,STREET_SUFFIX_DIR,STREET_NUMBER_SORT,MAILING_NEIGHBORHOOD,ZIP_CODE,X_COORD,Y_COORD,SAM_STREET_ID,WARD,PRECINCT_WARD,PARCEL
0,-71.125036547600004,42.250626552299998,1,1,100778,6-10 A St,6-10,1,6,10,...,,10,Hyde Park,02136,757684.428549561,2916575.26204486,1,18,1818,1809309000
1,-71.1248914926,42.250328423299997,2,1,126344,15 A St,15,0,NaN,NaN,...,,15,Hyde Park,02136,757724.178237994,2916466.7927758,1,18,1818,1809331000
2,-71.125400606,42.250468650199998,4,1,100797,7 A St,7,0,NaN,NaN,...,,7,Hyde Park,02136,757586.116749272,2916517.2874641,1,18,1818,1809337000
3,-71.056800588499996,42.340888670200002,11,1,154909,10 A St,10,0,NaN,NaN,...,,10,South Boston,02127,775987.55932641,2949556.68074288,2,06,0601,0600090000
4,-71.053148588300004,42.3448456709,16,1,141250,172-174 A St,172-174,1,172,174,...,,172,Boston,02210,776967.328447327,2951003.83399223,2,06,0601,0602743000


In [9]:
df_prop.head()

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,...,U_BTH_STYLE2,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW
0,0502550008,502550000,502550000,87,BEACON,ST,2-F,2108,102,CD,...,M,NaN,O,M,W,N,1,N,G,A
1,0502550010,502550000,502550000,87,BEACON,ST,2-R,2108,102,CD,...,NaN,NaN,O,S,W,N,1,N,A,A
2,0502550012,502550000,502550000,87,BEACON,ST,3-F,2108,102,CD,...,S,NaN,O,M,W,N,1,N,A,G
3,0502550014,502550000,502550000,87,BEACON,ST,3-R,2108,102,CD,...,NaN,NaN,O,N,W,N,1,S,A,G
4,0502550016,502550000,502550000,87,BEACON,ST,4,2108,102,CD,...,M,M,O,L,W,C,2,N,E,G


In [10]:
df_prop['ZIPCODE'] = '0' + df_prop['ZIPCODE']
df_prop['is_prop'] = 1
df_sam['is_sam'] = 1
df_sam['GIS_ID'] = df_sam['PARCEL'].copy().str[1:]

In [161]:
df_sam_unique = df_sam.groupby('GIS_ID').agg(
    {
        'X': 'first',
        'Y': 'first',
        'X_COORD': 'first',
        'Y_COORD': 'first',
        'is_sam': 'first',
        'is_sam': np.sum
    }
)

In [162]:
df_sam_unique.shape

(66783, 5)

In [163]:
df_sam_unique.tail()

,X,Y,X_COORD,Y_COORD,is_sam
GIS_ID,,,,,
905095000,-71.118380606200006,42.290288658100003,759421.780110985,2931036.67317362,4
905097000,-71.118650606100005,42.290328658100002,759348.661454916,2931050.92306164,4
905098000,-71.118900606099999,42.290338658499998,759281.003054842,2931054.26504564,1
905100000,-71.1184306058,42.290038658199997,759408.659342974,2930945.50888555,4
905101000,-71.118450605800007,42.289918658,759403.443566963,2930901.7547735,4


In [120]:
# df_merge = pd.merge(df_prop[['CM_ID', 'is_prop']], df_sam[['CM_ID', 'is_sam']], on='CM_ID', how='left')

In [164]:
df_merge = pd.merge(df_prop, df_sam_unique, on='GIS_ID', how='left')

In [165]:
df_merge.info()
df_merge.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174668 entries, 0 to 174667
Data columns (total 81 columns):
PID                174668 non-null object
CM_ID              82038 non-null object
GIS_ID             174668 non-null object
ST_NUM             174668 non-null object
ST_NAME            174668 non-null object
ST_NAME_SUF        174129 non-null object
UNIT_NUM           65700 non-null object
ZIPCODE            174661 non-null object
PTYPE              174668 non-null object
LU                 174668 non-null object
OWN_OCC            174668 non-null object
OWNER              174668 non-null object
MAIL_ADDRESSEE     174668 non-null object
MAIL_ADDRESS       174668 non-null object
MAIL CS            174668 non-null object
MAIL_ZIPCODE       174668 non-null object
AV_LAND            174668 non-null object
AV_BLDG            174668 non-null object
AV_TOTAL           174668 non-null object
GROSS_TAX          174668 non-null object
LAND_SF            169227 non-null object
YR_BUILT 

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,...,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW,is_prop,X,Y,X_COORD,Y_COORD,is_sam
0,0502550008,502550000,502550000,87,BEACON,ST,2-F,0000000002108,102,CD,...,1,N,G,A,1,-71.071690594499998,42.355918672,771934.51881063,2955013.33005996,16.0
1,0502550010,502550000,502550000,87,BEACON,ST,2-R,0000000002108,102,CD,...,1,N,A,A,1,-71.071690594499998,42.355918672,771934.51881063,2955013.33005996,16.0
2,0502550012,502550000,502550000,87,BEACON,ST,3-F,0000000002108,102,CD,...,1,N,A,G,1,-71.071690594499998,42.355918672,771934.51881063,2955013.33005996,16.0
3,0502550014,502550000,502550000,87,BEACON,ST,3-R,0000000002108,102,CD,...,1,S,A,G,1,-71.071690594499998,42.355918672,771934.51881063,2955013.33005996,16.0
4,0502550016,502550000,502550000,87,BEACON,ST,4,0000000002108,102,CD,...,2,N,E,G,1,-71.071690594499998,42.355918672,771934.51881063,2955013.33005996,16.0
5,0502551000,502551000,502551000,88,BEACON,ST,NaN,0000000002108,995,CM,...,NaN,NaN,NaN,NaN,1,-71.071790594600003,42.355898672,771907.526618615,2955005.90596394,7.0
6,0502551002,502551000,502551000,88,BEACON,ST,1,0000000002108,102,CD,...,0,N,A,F,1,-71.071790594600003,42.355898672,771907.526618615,2955005.90596394,7.0
7,0502551004,502551000,502551000,88,BEACON,ST,2,0000000002108,102,CD,...,1,N,A,F,1,-71.071790594600003,42.355898672,771907.526618615,2955005.90596394,7.0
8,0502551006,502551000,502551000,88,BEACON,ST,3,0000000002108,102,CD,...,2,E,E,E,1,-71.071790594600003,42.355898672,771907.526618615,2955005.90596394,7.0
9,0502551008,502551000,502551000,88,BEACON,ST,4,0000000002108,102,CD,...,0,N,A,E,1,-71.071790594600003,42.355898672,771907.526618615,2955005.90596394,7.0


In [166]:
df_merge['is_prop'].sum()

174668

In [167]:
df_merge[df_merge['is_sam'].isnull()]

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,...,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW,is_prop,X,Y,X_COORD,Y_COORD,is_sam
312,0100179002,NaN,100179002,,PRINCETON,ST,NaN,0000000002128,132,RL,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
441,0100289000,NaN,100289000,,PRINCETON,ST,NaN,0000000002128,106,RL,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
442,0100290000,NaN,100290000,,PRINCETON,ST,NaN,0000000002128,316,C,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
483,0100335000,NaN,100335000,,BREMEN,ST,NaN,0000000002128,337,CL,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
492,0100344000,NaN,100344000,368,BREMEN,ST,NaN,0000000002128,337,CL,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
497,0100352000,NaN,100352000,,NEPTUNE,RD,NaN,0000000002128,985,E,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
499,0100357000,NaN,100357000,36,NEPTUNE,RD,NaN,0000000002128,985,E,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
502,0100359001,NaN,100359001,405,FRANKFORT,ST,NaN,0000000002128,902,E,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
503,0100359002,NaN,100359002,409,FRANKFORT,ST,NaN,0000000002128,985,E,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
504,0100359005,NaN,100359005,,NEPTUNE,RD,NaN,0000000002128,901,E,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


In [147]:
len(df_merge.loc[df_merge[['is_prop', 'is_sam']].sum(axis=1)==2])

KeyError: "['is_sam'] not in index"

In [83]:
df_prop['ST_NUM'].value_counts()

              9445
1             2376
15            2337
10            2028
9             1958
2             1898
11            1804
8             1769
6             1721
25            1549
7             1532
12            1531
5             1515
19            1506
21            1478
20            1441
16            1405
14            1384
22            1375
18            1254
35            1253
3             1235
17            1224
4             1170
26            1129
24            1118
30            1113
27            1088
42            1047
40            1043
              ... 
55 W             1
425 A425         1
1110 A1108       1
51 - 53          1
199 -203         1
2 D              1
336 344          1
1 21             1
109 109A         1
41 -49           1
2521             1
1935             1
46 46R           1
1062 1064        1
695 A            1
401 405          1
24 2             1
662 A666A        1
53 HF            1
798 A            1
312 310          1
303 305B    

In [80]:
df_sam['STREET_NUMBER'].value_counts()

1            6649
10           5470
15           5224
11           5142
9            4221
6            4113
8            4085
20           3926
7            3898
5            3705
16           3642
25           3636
12           3633
14           3567
19           3559
30           3558
2            3494
4            3284
21           3171
40           3131
35           3048
24           3035
17           3002
3            2904
26           2849
18           2828
28           2806
31           2658
27           2613
22           2612
             ... 
554-560         1
874-880         1
3359-3365       1
57-6            1
3280-3288       1
580-590         1
511R            1
377-385         1
661B            1
232-236A        1
720A-720        1
1536-1538       1
25H             1
18B             1
119-119B        1
406-406         1
349-363         1
4446-4448       1
216-238         1
1866-1872       1
12 -14          1
8D              1
3134            1
1818            1
614-618   

In [86]:
df_sam['FULL_STREET_NAME']

0                 A St
1                 A St
2                 A St
3                 A St
4                 A St
5                 A St
6                 A St
7                 A St
8                 A St
9                 A St
10                A St
11                A St
12                A St
13                A St
14                A St
15                A St
16                A St
17                A St
18                A St
19                A St
20                A St
21                A St
22                A St
23                A St
24                A St
25                A St
26                A St
27                A St
28                A St
29                A St
              ...     
398672    Harrison Ave
398673    Harrison Ave
398674    Harrison Ave
398675    Harrison Ave
398676    Harrison Ave
398677    Harrison Ave
398678    Harrison Ave
398679    Harrison Ave
398680    Harrison Ave
398681    Harrison Ave
398682    Harrison Ave
398683    Harrison Ave
398684    H

In [95]:
df_sam.loc[df_sam['FULL_STREET_NAME'].str.contains('Beacon')]['STREET_NUMBER'].value_counts().sort_index().loc['87']

16

In [107]:
df_prop.loc[(df_prop['ST_NAME'].str.contains('BEACON')) & (df_prop['ST_NUM']=='87')]

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,...,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW,is_prop
0,0502550008,502550000,502550000,87,BEACON,ST,2-F,0000002108,102,CD,...,NaN,O,M,W,N,1,N,G,A,1
1,0502550010,502550000,502550000,87,BEACON,ST,2-R,0000002108,102,CD,...,NaN,O,S,W,N,1,N,A,A,1
2,0502550012,502550000,502550000,87,BEACON,ST,3-F,0000002108,102,CD,...,NaN,O,M,W,N,1,N,A,G,1
3,0502550014,502550000,502550000,87,BEACON,ST,3-R,0000002108,102,CD,...,NaN,O,N,W,N,1,S,A,G,1
4,0502550016,502550000,502550000,87,BEACON,ST,4,0000002108,102,CD,...,M,O,L,W,C,2,N,E,G,1
45818,0502550000,502550000,502550000,87,BEACON,ST,NaN,0000002108,995,CM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
45819,0502550002,502550000,502550000,87,BEACON,ST,G-1,0000002108,102,CD,...,NaN,O,M,W,N,0,N,G,A,1
45820,0502550004,502550000,502550000,87,BEACON,ST,1,0000002108,102,CD,...,NaN,O,M,W,C,1,N,G,A,1


In [141]:
df_sam.loc[(df_sam['FULL_STREET_NAME'].str.contains('Beacon')) & (df_sam['STREET_NUMBER']=='87')]#['ZIP_CODE']#[['STREET_NUMBER', 'PARCEL', 'CM_ID']]

,X,Y,SAM_ADDRESS_ID,RELATIONSHIP_TYPE,BUILDING_ID,FULL_ADDRESS,STREET_NUMBER,IS_RANGE,RANGE_FROM,RANGE_TO,...,ZIP_CODE,X_COORD,Y_COORD,SAM_STREET_ID,WARD,PRECINCT_WARD,PARCEL,is_sam,PID,CM_ID
8856,-71.071690594499998,42.355918672,12344,1,139636,87 Beacon St,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8857,-71.071690594499998,42.355918672,12345,2,139636,87 Beacon St #G1,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8858,-71.071690594499998,42.355918672,12346,2,139636,87 Beacon St #1,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8859,-71.071690594499998,42.355918672,12347,2,139636,87 Beacon St #2F,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8860,-71.071690594499998,42.355918672,12348,2,139636,87 Beacon St #2R,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8861,-71.071690594499998,42.355918672,12349,2,139636,87 Beacon St #3F,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8862,-71.071690594499998,42.355918672,12350,2,139636,87 Beacon St #3R,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
8863,-71.071690594499998,42.355918672,12351,2,139636,87 Beacon St #4,87,0,NaN,NaN,...,02108,771934.51881063,2955013.33005996,332,05,0505,0502550000,1,502550000,502550000
321165,-71.071690594700001,42.355918672199998,356456,2,139636,87 Beacon St #2,87,0,NaN,NaN,...,02108,771934.518766137,2955013.3301314,332,05,0505,0502550000,1,502550000,502550000
321166,-71.071690594700001,42.355918672199998,356457,2,139636,87 Beacon St #3,87,0,NaN,NaN,...,02108,771934.518766137,2955013.3301314,332,05,0505,0502550000,1,502550000,502550000


In [26]:
filepath

'../data/raw/bos_analyze/SAMaddresses.csv'

In [16]:
os.path.splitext(data_url)

('http://bostonopendata-boston.opendata.arcgis.com/datasets/b6bffcace320448d96bb84eabb8a075f_0',
 '.csv')

In [13]:

pathlib.Path(data_url).suffixes

['.csv']

In [ ]:
local_filename, headers = urllib.request.urlretrieve(
    'https://data.boston.gov/dataset/e02c44d2-3c64-459c-8fe2-e1ce5f38a035/resource/695a8596-5458-442b-a017-7cd72471aade/download/fy19fullpropassess.csv',
    '../data/raw/test_prop2.csv'
)